# 다이아몬드 데이터 분석 및 Cut 등급 예측
이 노트북은 다이아몬드 데이터셋을 분석하고, 여러 특성(캐럿, 색상, 투명도 등)을 사용하여 다이아몬드의 `cut` 등급을 예측하는 머신러닝 모델을 구축하는 과정을 보여줍니다.

**분석 과정:**
1.  데이터 로드 및 탐색 (EDA)
2.  범주형 데이터 전처리 (라벨 인코딩)
3.  모델 학습을 위한 데이터 준비 (특성, 타겟 분리)
4.  학습 및 테스트 데이터 분할
5.  로지스틱 회귀 모델 학습 및 평가

### 1. 라이브러리 임포트 및 데이터 로드

In [ ]:
import pandas as pd
import numpy as np

try:
    df = pd.read_csv("./data/diamonds.csv")
    print("데이터 로드 성공!")
except FileNotFoundError:
    print("오류: './data/diamonds.csv' 파일을 찾을 수 없습니다.")
    print("노트북이 있는 폴더(02_practice/0716) 아래에 'data/diamonds.csv' 파일을 위치시켜 주세요.")
    df = None

### 2. 데이터 탐색 (Exploratory Data Analysis - EDA)
데이터의 기본적인 정보를 확인합니다.

In [ ]:
if df is not None:
    print("데이터 샘플 (상위 5개):")
    print(df.head())
    print("
데이터 기본 정보:")
    print(df.info())
    print("
데이터 통계 요약:")
    print(df.describe())

#### 범주형 데이터 확인
`cut`, `color`, `clarity`와 같은 범주형(categorical) 데이터의 종류와 분포를 확인합니다.

In [ ]:
if df is not None:
    print("--- cut 변수 ---")
    print(df["cut"].value_counts())
    print("
--- color 변수 ---")
    print(df["color"].value_counts())
    print("
--- clarity 변수 ---")
    print(df["clarity"].value_counts())

### 3. 데이터 전처리: 라벨 인코딩
머신러닝 모델은 문자열 값을 직접 처리할 수 없으므로, `cut`, `color`, `clarity`와 같은 범주형 데이터를 숫자형으로 변환해야 합니다. 각 범주에 고유한 숫자를 부여하는 **라벨 인코딩(Label Encoding)**을 수행합니다.

In [ ]:
def get_label_map(field):
    """주어진 필드(컬럼)의 고유한 값들을 가져와 각 값에 숫자 라벨을 매핑하는 딕셔너리를 생성합니다."""
    unique_list = df[field].unique()
    label_map = {item: index + 1 for index, item in enumerate(unique_list)}
    return label_map

if df is not None:
    # 각 범주형 필드에 대해 라벨 인코딩 수행
    color_map = get_label_map('color')
    df['color_label'] = df['color'].map(color_map)

    clarity_map = get_label_map('clarity')
    df['clarity_label'] = df['clarity'].map(clarity_map)

    cut_map = get_label_map('cut')
    df['cut_label'] = df['cut'].map(cut_map)

    print("라벨 인코딩 후 데이터 샘플:")
    print(df.head())

### 4. 모델 학습을 위한 데이터 준비
모델에 입력으로 사용할 특성(X)과 예측 대상이 될 타겟(y)을 분리합니다.

- **특성 (X)**: `carat`, `depth`, `table`, `price`, `x`, `y`, `z` 및 라벨 인코딩된 `color_label`, `clarity_label`
- **타겟 (y)**: 예측하고자 하는 `cut_label`

In [ ]:
if df is not None:
    X = df.loc[:, ['carat', 'depth', 'table', 'price', 'x', 'y', 'z', 'color_label', 'clarity_label']]
    y = df.loc[:, 'cut_label']

    print("--- 특성 데이터 (X) 샘플 ---")
    print(X.head())
    print("
--- 타겟 데이터 (y) 샘플 ---")
    print(y.head())

### 5. 학습 및 테스트 데이터 분할
모델의 성능을 객관적으로 평가하기 위해, 전체 데이터를 학습용(train)과 테스트용(test)으로 분리합니다. `random_state`를 설정하여 항상 동일한 방식으로 데이터가 분할되도록 합니다.

In [ ]:
from sklearn.model_selection import train_test_split

if 'X' in locals() and 'y' in locals():
    X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)
    print(f"학습 데이터 크기: {X_train.shape}, {y_train.shape}")
    print(f"테스트 데이터 크기: {X_test.shape}, {y_test.shape}")

### 6. 로지스틱 회귀 모델 학습 및 평가
분류 문제에 사용되는 로지스틱 회귀 모델을 학습시키고, 학습 데이터와 테스트 데이터에 대한 정확도를 각각 평가합니다.

In [ ]:
from sklearn.linear_model import LogisticRegression

if 'X_train' in locals():
    model = LogisticRegression(max_iter=1000) # 반복 횟수를 늘려 모델이 수렴하도록 돕습니다.
    model.fit(X_train, y_train)

    train_score = model.score(X_train, y_train)
    test_score = model.score(X_test, y_test)

    print(f"훈련 세트 정확도: {train_score:.4f}")
    print(f"테스트 세트 정확도: {test_score:.4f}")

### ※ 참고: 원본 스크립트의 주석 처리된 코드
원본 `.py` 스크립트에는 결측치 처리와 관련된 코드가 주석으로 남아있었습니다. 이 데이터셋에는 결측치가 없지만, 일반적인 데이터 분석 과정에서는 결측치 확인 및 처리가 매우 중요합니다.

```python
#결측치 확인 - 중간에 있을 수도 있으니까 
print(df.isna().sum())

#결측치 제거 - 대부분의 경우는 제거보다는 다른값으로 대체하는경우가 많다. 
df = df.dropna(how="any", axis=0) #NaN 값이 있는 행은 모두 삭제해라
```